# FDS Challenge: Improved Pokemon Battle Prediction

This notebook improves upon the starter notebook with:

1. **Bug fixes** - Corrected ensemble model evaluation
2. **Better error handling** - Robust file loading and validation
3. **Enhanced features** - More sophisticated battle analysis
4. **Hyperparameter tuning** - Optimized model parameters
5. **Better evaluation** - Feature importance and model diagnostics

Let's get started!

## 1. Setup and Data Loading

In [1]:
!pip install xgboost lightgbm catboost

In [6]:
import json
import pandas as pd
import numpy as np
import os
from pathlib import Path
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Verify data files exist
def load_jsonl_data(file_path: str) -> list:
    """Safely load JSONL data with error handling."""
    if not Path(file_path).exists():
        raise FileNotFoundError(f"Data file not found: {file_path}")
    
    data = []
    try:
        with open(file_path, 'r') as f:
            for line_num, line in enumerate(f, 1):
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Warning: Skipping malformed JSON at line {line_num}")
        return data
    except Exception as e:
        raise Exception(f"Error loading {file_path}: {str(e)}")

# Define paths
COMPETITION_NAME = 'fds-pokemon-battles-prediction-2025'
DATA_PATH = Path('../input') / COMPETITION_NAME

# Check if we're in Kaggle environment
if not DATA_PATH.exists():
    # Try local paths
    DATA_PATH = Path('.')
    print(f"Using local data path: {DATA_PATH.absolute()}")

train_file_path = DATA_PATH / 'train.jsonl'
test_file_path = DATA_PATH / 'test.jsonl'

# Load data
print("Loading training data...")
train_data = load_jsonl_data(train_file_path)
print(f"✓ Loaded {len(train_data)} training battles")

print("\nLoading test data...")
test_data = load_jsonl_data(test_file_path)
print(f"✓ Loaded {len(test_data)} test battles")

# Inspect first battle
print("\n--- First Battle Structure ---")
first_battle = train_data[0].copy()
first_battle['battle_timeline'] = first_battle.get('battle_timeline', [])[:2]
print(json.dumps(first_battle, indent=2))
print("...")

Using local data path: /home/leyla/FDS-pokemon-challenge
Loading training data...
✓ Loaded 10000 training battles

Loading test data...
✓ Loaded 10000 training battles

Loading test data...
✓ Loaded 5000 test battles

--- First Battle Structure ---
{
  "player_won": true,
  "p1_team_details": [
    {
      "name": "starmie",
      "level": 100,
      "types": [
        "psychic",
        "water"
      ],
      "base_hp": 60,
      "base_atk": 75,
      "base_def": 85,
      "base_spa": 100,
      "base_spd": 100,
      "base_spe": 115
    },
    {
      "name": "exeggutor",
      "level": 100,
      "types": [
        "grass",
        "psychic"
      ],
      "base_hp": 95,
      "base_atk": 95,
      "base_def": 85,
      "base_spa": 125,
      "base_spd": 125,
      "base_spe": 55
    },
    {
      "name": "chansey",
      "level": 100,
      "types": [
        "normal",
        "notype"
      ],
      "base_hp": 250,
      "base_atk": 5,
      "base_def": 5,
      "base_spa": 105,


## 2. Data Validation and Exploration

In [3]:
# Check class balance
if 'player_won' in train_data[0]:
    y_values = [b['player_won'] for b in train_data]
    class_dist = pd.Series(y_values).value_counts()
    print("Class Distribution:")
    print(class_dist)
    print(f"\nBalance ratio: {class_dist.min() / class_dist.max():.2%}")
    
    if class_dist.min() / class_dist.max() < 0.8:
        print("⚠️  Warning: Classes are imbalanced - consider using stratified CV")
    else:
        print("✓ Classes are well balanced")

# Check for missing data
print("\n--- Data Completeness Check ---")
for key in ['battle_id', 'p1_team_details', 'p2_lead_details', 'battle_timeline']:
    missing = sum(1 for b in train_data if key not in b or not b[key])
    print(f"{key}: {len(train_data) - missing}/{len(train_data)} complete")

Class Distribution:
True     5000
False    5000
Name: count, dtype: int64

Balance ratio: 100.00%
✓ Classes are well balanced

--- Data Completeness Check ---
battle_id: 9999/10000 complete
p1_team_details: 10000/10000 complete
p2_lead_details: 10000/10000 complete
battle_timeline: 10000/10000 complete


## 3. Enhanced Feature Engineering

In [10]:
# Pokemon type data
pokedex = {
    "bulbasaur": ("grass", "poison"), "ivysaur": ("grass", "poison"), "venusaur": ("grass", "poison"),
    "charmander": ("fire", "notype"), "charmeleon": ("fire", "notype"), "charizard": ("fire", "flying"),
    "squirtle": ("water", "notype"), "wartortle": ("water", "notype"), "blastoise": ("water", "notype"),
    "caterpie": ("bug", "notype"), "metapod": ("bug", "notype"), "butterfree": ("bug", "flying"),
    "weedle": ("bug", "poison"), "kakuna": ("bug", "poison"), "beedrill": ("bug", "poison"),
    "pidgey": ("normal", "flying"), "pidgeotto": ("normal", "flying"), "pidgeot": ("normal", "flying"),
    "rattata": ("normal", "notype"), "raticate": ("normal", "notype"),
    "spearow": ("normal", "flying"), "fearow": ("normal", "flying"),
    "ekans": ("poison", "notype"), "arbok": ("poison", "notype"),
    "pikachu": ("electric", "notype"), "raichu": ("electric", "notype"),
    "sandshrew": ("ground", "notype"), "sandslash": ("ground", "notype"),
    "nidoran♀": ("poison", "notype"), "nidorina": ("poison", "notype"), "nidoqueen": ("poison", "ground"),
    "nidoran♂": ("poison", "notype"), "nidorino": ("poison", "notype"), "nidoking": ("poison", "ground"),
    "clefairy": ("normal", "notype"), "clefable": ("normal", "notype"),
    "vulpix": ("fire", "notype"), "ninetales": ("fire", "notype"),
    "jigglypuff": ("normal", "notype"), "wigglytuff": ("normal", "notype"),
    "zubat": ("poison", "flying"), "golbat": ("poison", "flying"),
    "oddish": ("grass", "poison"), "gloom": ("grass", "poison"), "vileplume": ("grass", "poison"),
    "paras": ("bug", "grass"), "parasect": ("bug", "grass"),
    "venonat": ("bug", "poison"), "venomoth": ("bug", "poison"),
    "diglett": ("ground", "notype"), "dugtrio": ("ground", "notype"),
    "meowth": ("normal", "notype"), "persian": ("normal", "notype"),
    "psyduck": ("water", "notype"), "golduck": ("water", "notype"),
    "mankey": ("fighting", "notype"), "primeape": ("fighting", "notype"),
    "growlithe": ("fire", "notype"), "arcanine": ("fire", "notype"),
    "poliwag": ("water", "notype"), "poliwhirl": ("water", "notype"), "poliwrath": ("water", "fighting"),
    "abra": ("psychic", "notype"), "kadabra": ("psychic", "notype"), "alakazam": ("psychic", "notype"),
    "machop": ("fighting", "notype"), "machoke": ("fighting", "notype"), "machamp": ("fighting", "notype"),
    "bellsprout": ("grass", "poison"), "weepinbell": ("grass", "poison"), "victreebel": ("grass", "poison"),
    "tentacool": ("water", "poison"), "tentacruel": ("water", "poison"),
    "geodude": ("rock", "ground"), "graveler": ("rock", "ground"), "golem": ("rock", "ground"),
    "ponyta": ("fire", "notype"), "rapidash": ("fire", "notype"),
    "slowpoke": ("water", "psychic"), "slowbro": ("water", "psychic"),
    "magnemite": ("electric", "notype"), "magneton": ("electric", "notype"),
    "farfetch'd": ("normal", "flying"), "doduo": ("normal", "flying"), "dodrio": ("normal", "flying"),
    "seel": ("water", "notype"), "dewgong": ("water", "ice"),
    "grimer": ("poison", "notype"), "muk": ("poison", "notype"),
    "shellder": ("water", "notype"), "cloyster": ("water", "ice"),
    "gastly": ("ghost", "poison"), "haunter": ("ghost", "poison"), "gengar": ("ghost", "poison"),
    "onix": ("rock", "ground"), "drowzee": ("psychic", "notype"), "hypno": ("psychic", "notype"),
    "krabby": ("water", "notype"), "kingler": ("water", "notype"),
    "voltorb": ("electric", "notype"), "electrode": ("electric", "notype"),
    "exeggcute": ("grass", "psychic"), "exeggutor": ("grass", "psychic"),
    "cubone": ("ground", "notype"), "marowak": ("ground", "notype"),
    "hitmonlee": ("fighting", "notype"), "hitmonchan": ("fighting", "notype"),
    "lickitung": ("normal", "notype"), "koffing": ("poison", "notype"), "weezing": ("poison", "notype"),
    "rhyhorn": ("ground", "rock"), "rhydon": ("ground", "rock"),
    "chansey": ("normal", "notype"), "tangela": ("grass", "notype"), "kangaskhan": ("normal", "notype"),
    "horsea": ("water", "notype"), "seadra": ("water", "notype"),
    "goldeen": ("water", "notype"), "seaking": ("water", "notype"),
    "staryu": ("water", "notype"), "starmie": ("water", "psychic"),
    "mr. mime": ("psychic", "notype"), "scyther": ("bug", "flying"),
    "jynx": ("ice", "psychic"), "electabuzz": ("electric", "notype"),
    "magmar": ("fire", "notype"), "pinsir": ("bug", "notype"),
    "tauros": ("normal", "notype"), "magikarp": ("water", "notype"),
    "gyarados": ("water", "flying"), "lapras": ("water", "ice"),
    "ditto": ("normal", "notype"), "eevee": ("normal", "notype"),
    "vaporeon": ("water", "notype"), "jolteon": ("electric", "notype"), "flareon": ("fire", "notype"),
    "porygon": ("normal", "notype"), "omanyte": ("rock", "water"), "omastar": ("rock", "water"),
    "kabuto": ("rock", "water"), "kabutops": ("rock", "water"),
    "aerodactyl": ("rock", "flying"), "snorlax": ("normal", "notype"),
    "articuno": ("ice", "flying"), "zapdos": ("electric", "flying"), "moltres": ("fire", "flying"),
    "dratini": ("dragon", "notype"), "dragonair": ("dragon", "notype"), "dragonite": ("dragon", "flying"),
    "mewtwo": ("psychic", "notype"), "mew": ("psychic", "notype"),
}

super_effective = {
    "normal": [], "fire": ["grass", "ice", "bug"], "water": ["fire", "ground", "rock"],
    "electric": ["water", "flying"], "grass": ["water", "ground", "rock"],
    "ice": ["grass", "ground", "flying", "dragon"], "fighting": ["normal", "ice", "rock"],
    "poison": ["grass", "bug"], "ground": ["fire", "electric", "poison", "rock"],
    "flying": ["grass", "fighting", "bug"], "psychic": ["fighting", "poison"],
    "bug": ["grass", "psychic"], "rock": ["fire", "ice", "flying", "bug"],
    "ghost": ["ghost"], "dragon": ["dragon"],
}

print("✓ Type data loaded")
print(f"Total Pokemon in Pokedex: {len(pokedex)}")

✓ Type data loaded
Total Pokemon in Pokedex: 151


In [12]:
def has_type_advantage(pokemon1: str, pokemon2: str, pokedex: dict) -> int:
    """Returns type advantage score (-1, 0, 1)."""
    if pokemon1 not in pokedex or pokemon2 not in pokedex:
        return 0
    
    types1 = pokedex[pokemon1]
    types2 = pokedex[pokemon2]
    
    advantage = 0
    for t1 in types1:
        if t1 == "notype":
            continue
        for t2 in types2:
            if t2 == "notype":
                continue
            if t2 in super_effective.get(t1, []):
                advantage += 1
    
    # Also check reverse
    disadvantage = 0
    for t2 in types2:
        if t2 == "notype":
            continue
        for t1 in types1:
            if t1 == "notype":
                continue
            if t1 in super_effective.get(t2, []):
                disadvantage += 1
    
    return advantage - disadvantage

def create_enhanced_features(data: list[dict]) -> pd.DataFrame:
    """Extract comprehensive battle features with advanced engineering."""
    feature_list = []
    
    for battle in tqdm(data, desc="Extracting features"):
        features = {}
        
        # ==== NEW: Team composition features ====
        p1_team = battle.get('p1_team_details', [])
        p2_lead = battle.get('p2_lead_details', {})
        
        if p1_team:
            features['p1_team_size'] = len(p1_team)
            features['p1_type_diversity'] = get_type_diversity(p1_team)
            features['p1_avg_hp'] = calculate_avg_stat(p1_team, 'hp')
            features['p1_avg_atk'] = calculate_avg_stat(p1_team, 'atk')
            features['p1_avg_def'] = calculate_avg_stat(p1_team, 'def')
            features['p1_avg_spa'] = calculate_avg_stat(p1_team, 'spa')
            features['p1_avg_spd'] = calculate_avg_stat(p1_team, 'spd')
            features['p1_avg_spe'] = calculate_avg_stat(p1_team, 'spe')
            
            # Stat ratios (offensive vs defensive balance)
            avg_atk = features['p1_avg_atk']
            avg_def = features['p1_avg_def']
            features['p1_offensive_ratio'] = avg_atk / avg_def if avg_def > 0 else 1
            features['p1_bulk'] = features['p1_avg_hp'] * features['p1_avg_def']
        
        if p2_lead:
            features['p2_lead_hp'] = p2_lead.get('base_hp', 0)
            features['p2_lead_atk'] = p2_lead.get('base_atk', 0)
            features['p2_lead_def'] = p2_lead.get('base_def', 0)
            features['p2_lead_spe'] = p2_lead.get('base_spe', 0)
        
        # Battle timeline analysis
        battle_timeline = battle.get('battle_timeline', [])
        
        if battle_timeline:
            # Initialize counters
            p1_state_counter = p2_state_counter = 0
            p1_hp_advantage_counter = p1_null_move_counter = p2_null_move_counter = 0
            p1_faints = p2_faints = 0
            p1_type_advantage_counter = 0
            p1_effects = p2_effects = 0
            p1_boosts = p2_boosts = 0
            p1_tauros = p1_chansey = p2_tauros = p2_chansey = 0
            p1_frz = p2_frz = 0
            
            # Move counters
            p1_hyperbeam = p2_hyperbeam = 0
            p1_thunder_wave = p2_thunder_wave = 0
            p1_explosion = p2_explosion = 0
            p1_body_slam = p2_body_slam = 0
            p1_rest = p2_rest = 0
            
            # Early game metrics
            p1_early_faints = p2_early_faints = 0
            p1_early_hp_advantage = 0
            first_blood = 0
            
            # NEW: Battle flow metrics
            total_turns = len(battle_timeline)
            p1_momentum = []  # Track HP advantage over time
            
            # NEW: Advanced counters
            p1_switches = p2_switches = 0
            p1_critical_hits = p2_critical_hits = 0
            p1_missed_moves = p2_missed_moves = 0
            p1_super_effective_hits = p2_super_effective_hits = 0
            
            # NEW: Status move tracking
            p1_status_moves = count_status_moves(battle_timeline, 'p1')
            p2_status_moves = count_status_moves(battle_timeline, 'p2')
            
            # NEW: Momentum shifts
            momentum_shifts = 0
            last_hp_diff = 0
            
            # NEW: Late game tracking (last 10 turns)
            p1_late_hp_advantage = 0
            
            for turn_idx, battle_step in enumerate(battle_timeline):
                p1_state = battle_step.get('p1_pokemon_state', {})
                p2_state = battle_step.get('p2_pokemon_state', {})
                
                # Type advantage
                p1_name = p1_state.get('name', '')
                p2_name = p2_state.get('name', '')
                if p1_name and p2_name:
                    type_adv = has_type_advantage(p1_name, p2_name, pokedex)
                    if type_adv > 0:
                        p1_type_advantage_counter += 1
                
                # Status conditions
                p1_status = p1_state.get('status', '')
                p2_status = p2_state.get('status', '')
                
                if p1_status and p1_status != "fnt":
                    p1_state_counter += 1
                    if p1_status == "frz":
                        p1_frz += 1
                        
                if p2_status and p2_status != "fnt":
                    p2_state_counter += 1
                    if p2_status == "frz":
                        p2_frz += 1
                
                # HP tracking
                p1_hp_pct = p1_state.get('hp_pct', 0)
                p2_hp_pct = p2_state.get('hp_pct', 0)
                
                if p1_hp_pct and p2_hp_pct:
                    hp_diff = p1_hp_pct - p2_hp_pct
                    p1_momentum.append(hp_diff)
                    if hp_diff > 0:
                        p1_hp_advantage_counter += 1
                    
                    # NEW: Track momentum shifts
                    if last_hp_diff != 0 and (hp_diff * last_hp_diff < 0):
                        momentum_shifts += 1
                    last_hp_diff = hp_diff
                
                # Faints
                if p1_status == "fnt":
                    p1_faints += 1
                if p2_status == "fnt":
                    p2_faints += 1
                
                # Effects and boosts
                if p1_state.get('effects') and "noeffect" not in p1_state.get('effects', []):
                    p1_effects += 1
                if p2_state.get('effects') and "noeffect" not in p2_state.get('effects', []):
                    p2_effects += 1
                
                p1_boosts_dict = p1_state.get('boosts', {})
                p2_boosts_dict = p2_state.get('boosts', {})
                if any(v > 0 for v in p1_boosts_dict.values()):
                    p1_boosts += 1
                if any(v > 0 for v in p2_boosts_dict.values()):
                    p2_boosts += 1
                
                # Pokemon counters
                if p1_name == "tauros":
                    p1_tauros += 1
                if p1_name == "chansey":
                    p1_chansey += 1
                if p2_name == "tauros":
                    p2_tauros += 1
                if p2_name == "chansey":
                    p2_chansey += 1
                
                # Move analysis
                p1_move = battle_step.get('p1_move_details', {})
                p2_move = battle_step.get('p2_move_details', {})
                
                if not p1_move:
                    p1_null_move_counter += 1
                else:
                    move_name = p1_move.get('name', '')
                    if move_name == 'hyperbeam':
                        p1_hyperbeam += 1
                    elif move_name == 'bodyslam':
                        p1_body_slam += 1
                    elif move_name == 'thunderwave':
                        p1_thunder_wave += 1
                    elif move_name in ['explosion', 'selfdestruct']:
                        p1_explosion += 1
                    elif move_name == 'rest':
                        p1_rest += 1
                    
                    # NEW: Track switches
                    if move_name == 'switch':
                        p1_switches += 1
                
                if not p2_move:
                    p2_null_move_counter += 1
                else:
                    move_name = p2_move.get('name', '')
                    if move_name == 'hyperbeam':
                        p2_hyperbeam += 1
                    elif move_name == 'bodyslam':
                        p2_body_slam += 1
                    elif move_name == 'thunderwave':
                        p2_thunder_wave += 1
                    elif move_name in ['explosion', 'selfdestruct']:
                        p2_explosion += 1
                    elif move_name == 'rest':
                        p2_rest += 1
                    
                    # NEW: Track switches
                    if move_name == 'switch':
                        p2_switches += 1
                
                # Early game tracking
                if turn_idx < 10:
                    if p1_status == "fnt":
                        p1_early_faints += 1
                        if first_blood == 0:
                            first_blood = -1
                    if p2_status == "fnt":
                        p2_early_faints += 1
                        if first_blood == 0:
                            first_blood = 1
                    if p1_hp_pct and p2_hp_pct and p1_hp_pct > p2_hp_pct:
                        p1_early_hp_advantage += 1
                
                # NEW: Late game tracking
                if turn_idx >= total_turns - 10:
                    if p1_hp_pct and p2_hp_pct and p1_hp_pct > p2_hp_pct:
                        p1_late_hp_advantage += 1
            
            # Store all features
            features['total_turns'] = total_turns
            features['p1_state_counter'] = p1_state_counter
            features['p2_state_counter'] = p2_state_counter
            features['p1_hp_advantage_counter'] = p1_hp_advantage_counter
            features['p1_null_move_counter'] = p1_null_move_counter
            features['p2_null_move_counter'] = p2_null_move_counter
            features['p1_faints_counter'] = p1_faints
            features['p2_faints_counter'] = p2_faints
            features['p1_type_advantage_counter'] = p1_type_advantage_counter
            features['p1_effects_counter'] = p1_effects
            features['p2_effects_counter'] = p2_effects
            features['p1_boosts_counter'] = p1_boosts
            features['p2_boosts_counter'] = p2_boosts
            features['p1_chansey_counter'] = p1_chansey
            features['p1_tauros_counter'] = p1_tauros
            features['p2_chansey_counter'] = p2_chansey
            features['p2_tauros_counter'] = p2_tauros
            features['p1_early_faints'] = p1_early_faints
            features['p2_early_faints'] = p2_early_faints
            features['first_blood'] = first_blood
            features['p1_early_hp_advantage'] = p1_early_hp_advantage
            features['p1_frz'] = p1_frz
            features['p2_frz'] = p2_frz
            features['p1_hyperbeam_counter'] = p1_hyperbeam
            features['p2_hyperbeam_counter'] = p2_hyperbeam
            features['p1_thunder_wave'] = p1_thunder_wave
            features['p2_thunder_wave'] = p2_thunder_wave
            features['p1_explosion_selfdestruct'] = p1_explosion
            features['p2_explosion_selfdestruct'] = p2_explosion
            features['p1_body_slam'] = p1_body_slam
            features['p2_body_slam'] = p2_body_slam
            features['p1_rest'] = p1_rest
            features['p2_rest'] = p2_rest
            
            # NEW: Advanced features
            features['p1_switches'] = p1_switches
            features['p2_switches'] = p2_switches
            features['momentum_shifts'] = momentum_shifts
            features['p1_late_hp_advantage'] = p1_late_hp_advantage
            
            # NEW: Status move features
            for status_type, count in p1_status_moves.items():
                features[f'p1_{status_type}_moves'] = count
            for status_type, count in p2_status_moves.items():
                features[f'p2_{status_type}_moves'] = count
            
            # Derived features
            features['faint_difference'] = p2_faints - p1_faints
            features['null_move_difference'] = p2_null_move_counter - p1_null_move_counter
            features['state_difference'] = p2_state_counter - p1_state_counter
            features['effects_difference'] = p2_effects - p1_effects
            features['switch_difference'] = p1_switches - p2_switches
            
            # Momentum feature
            if p1_momentum:
                features['avg_hp_momentum'] = np.mean(p1_momentum)
                features['momentum_trend'] = p1_momentum[-1] - p1_momentum[0] if len(p1_momentum) > 1 else 0
                features['momentum_volatility'] = np.std(p1_momentum) if len(p1_momentum) > 1 else 0
            else:
                features['avg_hp_momentum'] = 0
                features['momentum_trend'] = 0
                features['momentum_volatility'] = 0
            
            # NEW: Interaction features (high value combinations)
            features['faint_x_momentum'] = features['faint_difference'] * features['avg_hp_momentum']
            features['early_advantage_x_total_turns'] = features['p1_early_hp_advantage'] * total_turns
            features['freeze_advantage'] = (p2_frz - p1_frz) * 5  # Freeze is very impactful
        
        # Metadata
        features['battle_id'] = battle.get('battle_id')
        if 'player_won' in battle:
            features['player_won'] = int(battle['player_won'])
        
        feature_list.append(features)
    
    return pd.DataFrame(feature_list).fillna(0)

# Generate features
print("Processing training data with advanced features...")
train_df = create_enhanced_features(train_data)

print("\nProcessing test data...")
test_df = create_enhanced_features(test_data)

print("\n✓ Feature extraction complete")
print(f"Training shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

display(train_df.head())

Processing training data with advanced features...


Extracting features:   0%|          | 0/10000 [00:00<?, ?it/s]


Processing test data...


Extracting features:   0%|          | 0/5000 [00:00<?, ?it/s]


✓ Feature extraction complete
Training shape: (10000, 74)
Test shape: (5000, 73)


,p1_team_size,p1_type_diversity,p1_avg_hp,p1_avg_atk,p1_avg_def,p1_avg_spa,p1_avg_spd,p1_avg_spe,p1_offensive_ratio,p1_bulk,...,effects_difference,switch_difference,avg_hp_momentum,momentum_trend,momentum_volatility,faint_x_momentum,early_advantage_x_total_turns,freeze_advantage,battle_id,player_won
0,6,4,115.833333,72.500000,63.333333,100.000000,100.000000,80.000000,1.144737,7336.111111,...,-2,0,-0.031647,-0.030883,0.374985,-0.000000,150,55,0,1
1,6,5,123.333333,72.500000,65.833333,90.000000,90.000000,61.666667,1.101266,8119.444444,...,0,0,0.007778,-0.320000,0.322437,-0.023333,120,0,1,1
2,6,7,124.166667,84.166667,71.666667,90.000000,90.000000,65.833333,1.174419,8898.611111,...,3,0,0.084828,-0.240000,0.276650,-0.084828,180,0,2,1
3,6,7,121.666667,77.500000,65.833333,103.333333,103.333333,75.833333,1.177215,8009.722222,...,4,0,-0.011481,0.650000,0.359946,0.034444,120,0,3,1
4,6,5,114.166667,75.833333,79.166667,97.500000,97.500000,72.500000,0.957895,9038.194444,...,2,0,0.033103,0.280000,0.354051,-0.033103,30,0,4,1


### Advanced Feature Ideas

Before running the feature extraction, let's add more sophisticated features:
- **Team composition features** (type diversity, offensive/defensive balance)
- **Move effectiveness patterns** (STAB moves, coverage moves)
- **Stat-based features** (speed tiers, bulk calculations)
- **Sequential patterns** (momentum shifts, comeback indicators)
- **Interaction features** (cross-features between different metrics)

In [11]:
# Additional helper functions for advanced features

def get_type_diversity(team: list) -> float:
    """Calculate type diversity of a team."""
    if not team:
        return 0
    types_seen = set()
    for pokemon in team:
        name = pokemon.get('name', '')
        if name in pokedex:
            for ptype in pokedex[name]:
                if ptype != 'notype':
                    types_seen.add(ptype)
    return len(types_seen)

def calculate_avg_stat(team: list, stat_name: str) -> float:
    """Calculate average base stat for a team."""
    if not team:
        return 0
    total = sum(p.get(f'base_{stat_name}', 0) for p in team)
    return total / len(team)

def count_status_moves(battle_timeline: list, player: str) -> dict:
    """Count different types of status moves used."""
    status_moves = {
        'paralysis': ['thunderwave', 'stunspore', 'glare', 'bodyslam'],
        'sleep': ['sleeppowder', 'hypnosis', 'lovelykiss', 'sing', 'spore'],
        'poison': ['toxic', 'poisonpowder', 'poisongas'],
        'burn': ['willowisp', 'fireblast'],
        'setup': ['swordsdance', 'amnesia', 'agility', 'growth']
    }
    
    counts = {key: 0 for key in status_moves.keys()}
    
    for turn in battle_timeline:
        move_details = turn.get(f'{player}_move_details', {})
        if move_details:
            move_name = move_details.get('name', '')
            for category, moves in status_moves.items():
                if move_name in moves:
                    counts[category] += 1
    
    return counts

print("✓ Advanced helper functions loaded")

✓ Advanced helper functions loaded


## 4. Model Training with Cross-Validation

In [13]:
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Prepare data
features = [col for col in train_df.columns if col not in ['battle_id', 'player_won']]
X_train = train_df[features]
y_train = train_df['player_won']
X_test = test_df[features]

# Use stratified K-fold for better balance
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("✓ Data prepared for modeling")
print(f"Features: {len(features)}")
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

✓ Data prepared for modeling
Features: 72
Training samples: 10000
Test samples: 5000


In [14]:
# Define models
log_reg_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42, C=0.5))
])

svm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC(kernel='rbf', probability=True, random_state=42, C=1.0))
])

rf_model = RandomForestClassifier(n_estimators=300, max_depth=12, random_state=42, min_samples_split=5)

xgb_model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    random_state=42,
    eval_metric='logloss',
    verbosity=0
)

lgbm_model = LGBMClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    num_leaves=31,
    random_state=42,
    verbose=-1,
    force_col_wise=True
)

catboost_model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.05,
    depth=6,
    loss_function='Logloss',
    verbose=False,
    random_state=42
)

models = {
    'Logistic Regression': log_reg_pipe,
    'Random Forest': rf_model,
    'SVM': svm_pipe,
    'XGBoost': xgb_model,
    'LightGBM': lgbm_model,
    'CatBoost': catboost_model
}

model_results = {}

print("=" * 60)
print("INDIVIDUAL MODEL PERFORMANCE")
print("=" * 60)

for name, model in models.items():
    print(f"\nEvaluating {name}...")
    cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy')
    
    model_results[name] = {
        'model': model,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'cv_scores': cv_scores
    }
    
    print(f"{name}: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

print("\n✓ Individual model evaluation complete")

INDIVIDUAL MODEL PERFORMANCE

Evaluating Logistic Regression...
Logistic Regression: 0.8134 (+/- 0.0061)

Evaluating Random Forest...
Logistic Regression: 0.8134 (+/- 0.0061)

Evaluating Random Forest...
Random Forest: 0.8032 (+/- 0.0067)

Evaluating SVM...
Random Forest: 0.8032 (+/- 0.0067)

Evaluating SVM...
SVM: 0.8185 (+/- 0.0043)

Evaluating XGBoost...
SVM: 0.8185 (+/- 0.0043)

Evaluating XGBoost...
XGBoost: 0.8166 (+/- 0.0068)

Evaluating LightGBM...
XGBoost: 0.8166 (+/- 0.0068)

Evaluating LightGBM...
LightGBM: 0.8183 (+/- 0.0071)

Evaluating CatBoost...
LightGBM: 0.8183 (+/- 0.0071)

Evaluating CatBoost...
CatBoost: 0.8155 (+/- 0.0040)

✓ Individual model evaluation complete
CatBoost: 0.8155 (+/- 0.0040)

✓ Individual model evaluation complete


## 5. Ensemble Models

## 4.5. Hyperparameter Tuning (Optional but Recommended)

Let's optimize the best performing models with GridSearchCV or RandomizedSearchCV for better accuracy.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Choose top 2-3 models to tune
models_to_tune = ['Random Forest', 'XGBoost', 'LightGBM']

tuned_models = {}

print("=" * 60)
print("HYPERPARAMETER TUNING")
print("=" * 60)

# Random Forest tuning
if 'Random Forest' in models_to_tune:
    print("\n🔧 Tuning Random Forest...")
    rf_param_dist = {
        'n_estimators': [200, 300, 500],
        'max_depth': [10, 12, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2', None]
    }
    
    rf_random = RandomizedSearchCV(
        RandomForestClassifier(random_state=42),
        param_distributions=rf_param_dist,
        n_iter=20,
        cv=3,
        scoring='accuracy',
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    
    rf_random.fit(X_train, y_train)
    tuned_models['Random Forest'] = rf_random.best_estimator_
    print(f"✓ Best RF params: {rf_random.best_params_}")
    print(f"✓ Best RF score: {rf_random.best_score_:.4f}")

# XGBoost tuning
if 'XGBoost' in models_to_tune:
    print("\n🔧 Tuning XGBoost...")
    xgb_param_dist = {
        'n_estimators': [200, 300, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [4, 6, 8, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
    }
    
    xgb_random = RandomizedSearchCV(
        XGBClassifier(random_state=42, eval_metric='logloss', verbosity=0),
        param_distributions=xgb_param_dist,
        n_iter=20,
        cv=3,
        scoring='accuracy',
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    
    xgb_random.fit(X_train, y_train)
    tuned_models['XGBoost'] = xgb_random.best_estimator_
    print(f"✓ Best XGB params: {xgb_random.best_params_}")
    print(f"✓ Best XGB score: {xgb_random.best_score_:.4f}")

# LightGBM tuning
if 'LightGBM' in models_to_tune:
    print("\n🔧 Tuning LightGBM...")
    lgbm_param_dist = {
        'n_estimators': [200, 300, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [4, 6, 8, 10],
        'num_leaves': [15, 31, 63, 127],
        'min_child_samples': [10, 20, 30],
        'subsample': [0.7, 0.8, 0.9, 1.0]
    }
    
    lgbm_random = RandomizedSearchCV(
        LGBMClassifier(random_state=42, verbose=-1, force_col_wise=True),
        param_distributions=lgbm_param_dist,
        n_iter=20,
        cv=3,
        scoring='accuracy',
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    
    lgbm_random.fit(X_train, y_train)
    tuned_models['LightGBM'] = lgbm_random.best_estimator_
    print(f"✓ Best LGBM params: {lgbm_random.best_params_}")
    print(f"✓ Best LGBM score: {lgbm_random.best_score_:.4f}")

# Update model_results with tuned models
for name, tuned_model in tuned_models.items():
    cv_scores = cross_val_score(tuned_model, X_train, y_train, cv=kf, scoring='accuracy')
    model_results[f'{name} (Tuned)'] = {
        'model': tuned_model,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'cv_scores': cv_scores
    }
    print(f"\n{name} (Tuned): {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

print("\n✓ Hyperparameter tuning complete")

HYPERPARAMETER TUNING

🔧 Tuning Random Forest...
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Fitting 3 folds for each of 20 candidates, totalling 60 fits
✓ Best RF params: {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20}
✓ Best RF score: 0.8050

🔧 Tuning XGBoost...
Fitting 3 folds for each of 20 candidates, totalling 60 fits
✓ Best RF params: {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20}
✓ Best RF score: 0.8050

🔧 Tuning XGBoost...
Fitting 3 folds for each of 20 candidates, totalling 60 fits


### 4.6. Feature Selection (Remove Low-Importance Features)

Removing noisy or redundant features can sometimes improve model performance.

In [ ]:
from sklearn.feature_selection import SelectFromModel

# Use the best model (or Random Forest) to select features
print("=" * 60)
print("FEATURE SELECTION")
print("=" * 60)

# Train a Random Forest for feature importance
selector_model = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42)
selector_model.fit(X_train, y_train)

# Get feature importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': selector_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features:")
print(feature_importance.head(20).to_string(index=False))

# Select features with importance above threshold
selector = SelectFromModel(selector_model, threshold='median', prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

selected_features = [f for f, selected in zip(features, selector.get_support()) if selected]
print(f"\n✓ Selected {len(selected_features)} features out of {len(features)}")
print(f"Reduction: {(1 - len(selected_features)/len(features))*100:.1f}%")

# Test if feature selection improves accuracy
print("\n--- Testing with Selected Features ---")
rf_selected = RandomForestClassifier(n_estimators=300, max_depth=12, random_state=42)
cv_scores_selected = cross_val_score(rf_selected, X_train_selected, y_train, cv=kf, scoring='accuracy')
print(f"RF with selected features: {cv_scores_selected.mean():.4f} (+/- {cv_scores_selected.std():.4f})")

# Compare to using all features
cv_scores_all = cross_val_score(rf_model, X_train, y_train, cv=kf, scoring='accuracy')
print(f"RF with all features:      {cv_scores_all.mean():.4f} (+/- {cv_scores_all.std():.4f})")

if cv_scores_selected.mean() > cv_scores_all.mean():
    print("\n✅ Feature selection improved performance! Consider using selected features.")
    # Optionally update X_train and X_test
    # X_train = X_train_selected
    # X_test = X_test_selected
    # features = selected_features
else:
    print("\n⚠️  Feature selection did not improve performance. Keep all features.")

### 4.7. Polynomial Features (Advanced - Optional)

Create interaction terms between important features for non-linear relationships.

In [ ]:
# WARNING: This can significantly increase feature count and training time
# Only use on selected important features

from sklearn.preprocessing import PolynomialFeatures

# Select most important features for polynomial expansion
important_features = [
    'faint_difference', 'avg_hp_momentum', 'p1_hp_advantage_counter',
    'p1_early_hp_advantage', 'total_turns', 'first_blood',
    'null_move_difference', 'state_difference', 'momentum_trend'
]

# Only expand if these features exist
available_important = [f for f in important_features if f in X_train.columns]

if len(available_important) >= 5:
    print("=" * 60)
    print("POLYNOMIAL FEATURE EXPANSION")
    print("=" * 60)
    
    # Create polynomial features (degree 2) for selected features
    poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
    
    X_train_important = X_train[available_important]
    X_test_important = X_test[available_important]
    
    X_train_poly = poly.fit_transform(X_train_important)
    X_test_poly = poly.transform(X_test_important)
    
    print(f"Original features: {len(available_important)}")
    print(f"After polynomial expansion: {X_train_poly.shape[1]}")
    
    # Combine with original features
    X_train_combined = np.hstack([X_train.values, X_train_poly])
    X_test_combined = np.hstack([X_test.values, X_test_poly])
    
    # Test with Random Forest
    print("\n--- Testing with Polynomial Features ---")
    rf_poly = RandomForestClassifier(n_estimators=200, max_depth=12, random_state=42)
    cv_scores_poly = cross_val_score(rf_poly, X_train_combined, y_train, cv=3, scoring='accuracy')
    print(f"RF with polynomial features: {cv_scores_poly.mean():.4f} (+/- {cv_scores_poly.std():.4f})")
    
    cv_scores_original = cross_val_score(rf_model, X_train, y_train, cv=3, scoring='accuracy')
    print(f"RF with original features:   {cv_scores_original.mean():.4f} (+/- {cv_scores_original.std():.4f})")
    
    if cv_scores_poly.mean() > cv_scores_original.mean():
        print("\n✅ Polynomial features improved performance!")
    else:
        print("\n⚠️  Polynomial features did not improve performance. Skip this step.")
else:
    print("⚠️  Not enough important features available for polynomial expansion.")

In [ ]:
print("=" * 60)
print("VOTING ENSEMBLE")
print("=" * 60)

voting_ensemble = VotingClassifier(
    estimators=[
        ('lr', log_reg_pipe),
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('lgbm', lgbm_model),
        ('catboost', catboost_model)
    ],
    voting='soft'
)

cv_scores = cross_val_score(voting_ensemble, X_train, y_train, cv=kf, scoring='accuracy')

# BUG FIX: Use cv_scores instead of scores
model_results['Voting Ensemble'] = {
    'model': voting_ensemble,
    'cv_mean': cv_scores.mean(),
    'cv_std': cv_scores.std(),
    'cv_scores': cv_scores
}

print(f"Voting Ensemble: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
print(f"Individual Fold Scores: {cv_scores.round(4)}")

VOTING ENSEMBLE
Voting Ensemble: 0.8096 (+/- 0.0081)
Individual Fold Scores: [0.7985 0.8235 0.807  0.811  0.808 ]


In [ ]:
print("\n" + "=" * 60)
print("STACKING ENSEMBLE")
print("=" * 60)

stacking_ensemble = StackingClassifier(
    estimators=[
        ('lr', log_reg_pipe),
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('lgbm', lgbm_model),
        ('catboost', catboost_model)
    ],
    final_estimator=LogisticRegression(max_iter=1000, random_state=42),
    cv=5
)

cv_scores = cross_val_score(stacking_ensemble, X_train, y_train, cv=kf, scoring='accuracy')

# BUG FIX: Use cv_scores instead of scores
model_results['Stacking Ensemble'] = {
    'model': stacking_ensemble,
    'cv_mean': cv_scores.mean(),
    'cv_std': cv_scores.std(),
    'cv_scores': cv_scores
}

print(f"Stacking Ensemble: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
print(f"Individual Fold Scores: {cv_scores.round(4)}")


STACKING ENSEMBLE
Stacking Ensemble: 0.8109 (+/- 0.0064)
Individual Fold Scores: [0.8025 0.822  0.808  0.8115 0.8105]


## 6. Model Selection and Analysis

In [ ]:
print("\n" + "=" * 60)
print("MODEL COMPARISON")
print("=" * 60)

# Sort by CV accuracy
sorted_models = sorted(model_results.items(), key=lambda x: x[1]['cv_mean'], reverse=True)

print(f"\n{'Rank':<5} {'Model':<25} {'CV Accuracy':<15} {'Std Dev'}")
print("-" * 60)

for rank, (name, results) in enumerate(sorted_models, 1):
    print(f"{rank:<5} {name:<25} {results['cv_mean']:.4f}        (+/- {results['cv_std']:.4f})")

# Select best model
best_model_name, best_results = sorted_models[0]
print(f"\n{'='*60}")
print(f"🏆 BEST MODEL: {best_model_name}")
print(f"   CV Accuracy: {best_results['cv_mean']:.4f} (+/- {best_results['cv_std']:.4f})")
print(f"{'='*60}")


MODEL COMPARISON

Rank  Model                     CV Accuracy     Std Dev
------------------------------------------------------------
1     CatBoost                  0.8114        (+/- 0.0056)
2     SVM                       0.8114        (+/- 0.0085)
3     Stacking Ensemble         0.8109        (+/- 0.0064)
4     Voting Ensemble           0.8096        (+/- 0.0081)
5     LightGBM                  0.8091        (+/- 0.0077)
6     Logistic Regression       0.8075        (+/- 0.0020)
7     XGBoost                   0.8064        (+/- 0.0109)
8     Random Forest             0.8030        (+/- 0.0079)

🏆 BEST MODEL: CatBoost
   CV Accuracy: 0.8114 (+/- 0.0056)


In [ ]:
# Feature importance (if available)
best_model = best_results['model']

# Train on full dataset to extract feature importance
print(f"\nTraining {best_model_name} on full training set...")
best_model.fit(X_train, y_train)
print("✓ Training complete")

# Extract feature importance if available
try:
    if hasattr(best_model, 'feature_importances_'):
        importances = best_model.feature_importances_
    elif hasattr(best_model, 'named_steps') and hasattr(best_model.named_steps['classifier'], 'coef_'):
        importances = np.abs(best_model.named_steps['classifier'].coef_[0])
    elif hasattr(best_model, 'final_estimator_') and hasattr(best_model.final_estimator_, 'coef_'):
        importances = np.abs(best_model.final_estimator_.coef_[0])
    else:
        importances = None
    
    if importances is not None:
        feature_importance_df = pd.DataFrame({
            'feature': features,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        print("\n" + "=" * 60)
        print("TOP 15 MOST IMPORTANT FEATURES")
        print("=" * 60)
        print(feature_importance_df.head(15).to_string(index=False))
except Exception as e:
    print(f"\n⚠️  Could not extract feature importance: {str(e)}")


Training CatBoost on full training set...
✓ Training complete

TOP 15 MOST IMPORTANT FEATURES
                  feature  importance
     null_move_difference   12.063095
         state_difference    6.774463
         faint_difference    6.479559
        p2_tauros_counter    5.399867
           momentum_trend    5.009113
          avg_hp_momentum    4.960013
         p1_state_counter    4.213907
        p1_faints_counter    4.030504
     p2_null_move_counter    3.848861
       p2_chansey_counter    3.687638
                   p1_frz    2.818481
p1_explosion_selfdestruct    2.581571
        p1_tauros_counter    2.479417
                   p2_frz    2.464095
p1_type_advantage_counter    2.324548


## 7. Generate Predictions

In [ ]:
print("\n" + "=" * 60)
print("GENERATING PREDICTIONS")
print("=" * 60)

y_pred = best_model.predict(X_test)

# Create submission
submission_df = pd.DataFrame({
    'battle_id': test_df['battle_id'],
    'player_won': y_pred
})

submission_df.to_csv('submission.csv', index=False)

print(f"\n✓ Predictions saved to 'submission.csv'")
print(f"Total predictions: {len(submission_df)}")
print(f"\nPrediction distribution:")
print(f"  Player won (1): {(y_pred == 1).sum():,} ({(y_pred == 1).sum()/len(y_pred)*100:.1f}%)")
print(f"  Player lost (0): {(y_pred == 0).sum():,} ({(y_pred == 0).sum()/len(y_pred)*100:.1f}%)")

print("\n--- Sample Predictions ---")
display(submission_df.head(10))


GENERATING PREDICTIONS

✓ Predictions saved to 'submission.csv'
Total predictions: 5000

Prediction distribution:
  Player won (1): 2,502 (50.0%)
  Player lost (0): 2,498 (50.0%)

--- Sample Predictions ---


,battle_id,player_won
0,0,0
1,1,1
2,2,1
3,3,1
4,4,0
5,5,0
6,6,1
7,7,1
8,8,1
9,9,1


## 8. Summary

### Key Improvements Made:

1. **Bug Fixes**
   - Fixed ensemble model evaluation (voting & stacking)
   - Proper error handling for file operations

2. **Enhanced Features** ⭐ NEW
   - **Team composition**: Type diversity, average stats, offensive/defensive ratios
   - **Advanced battle metrics**: Switches, momentum shifts, late-game performance
   - **Status move tracking**: Paralysis, sleep, poison, setup moves
   - **Interaction features**: Cross-features between important metrics
   - Total turn count, HP momentum tracking, momentum volatility
   - Improved type advantage calculation

3. **Better Modeling**
   - Stratified K-fold cross-validation
   - **Hyperparameter tuning** with RandomizedSearchCV
   - **Feature selection** to remove noise
   - Feature importance analysis

4. **Code Quality**
   - Robust data loading
   - Data validation checks
   - Better progress tracking
   - Clear output formatting

### Additional Techniques to Try:

#### 1. **Feature Engineering Ideas**
   - Team synergy features (type coverage analysis)
   - Speed tier advantages (who moves first)
   - Move coverage scores (how well team covers opponent weaknesses)
   - Pokemon role detection (sweeper, tank, support)
   - Sequence patterns (common move combos)

#### 2. **Model Ensembling Strategies**
   - Weighted voting based on CV performance
   - Blending predictions from different feature sets
   - Two-level stacking (meta-meta model)

#### 3. **Advanced ML Techniques**
   - **Gradient Boosting with custom objectives**: XGBoost/LightGBM with focal loss
   - **Calibrated classifiers**: Improve probability estimates
   - **Optuna/Bayesian optimization**: Smarter hyperparameter search
   - **TabNet**: Attention-based tabular model (not quite neural network)

#### 4. **Data Augmentation & Preprocessing**
   - Create synthetic battles by swapping P1/P2
   - Polynomial features for key interactions
   - Target encoding for categorical features
   - Outlier detection and removal

#### 5. **Cross-Validation Strategies**
   - Time-based splits if battles have temporal ordering
   - Group K-fold if multiple battles from same players
   - Stratified by battle length or difficulty

#### 6. **Post-Processing**
   - Threshold optimization (adjust decision boundary)
   - Ensemble calibration
   - Analyze and fix systematic errors

### Quick Wins for Accuracy Boost:

1. ✅ **Run hyperparameter tuning** (section 4.5) - Can gain 0.5-2%
2. ✅ **Add team composition features** (already added) - High impact
3. ✅ **Use best ensemble** (voting/stacking) - Gains 0.3-1%
4. 🔄 **Feature engineering**: Add Pokemon role/tier features
5. 🔄 **Calibrate predictions** for better probability estimates

### Expected Accuracy Improvements:

- **Current baseline**: ~70-75% (simple features)
- **With enhanced features**: ~75-80% (+3-5%)
- **With hyperparameter tuning**: ~78-82% (+3-4%)
- **With optimal ensemble**: ~80-85% (+2-3%)
- **Ceiling (without neural nets)**: ~85-88%

Good luck with your submission! 🚀